<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Рекомендация тарифов

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф.

Для работы выданы данные о поведении клиентов, которые уже перешли на тарифы провайдера. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier 
import numpy as np

In [2]:
try:
    df = pd.read_csv('datasets/users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# изучим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков
- minutes — суммарная длительность звонков в минутах
- messages — количество sms-сообщений
- mb_used — израсходованный интернет-трафик в Мб
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Разбейте данные на выборки

Итак, чтобы лучше оценить обученную модель (нет ли переобучения и тд) нужны не только обучающая и тестовая, но и валидационная выборка.

Поэтому нужно разбить данные на три части: обучающая(60%), валидационная(20%) и тестовая(20%).

In [4]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

# сначала отделим обучающую выборку
features_train, features_valid_and_test, target_train, target_valid_and_test = train_test_split(features, 
                                                                                                target, test_size=0.4, 
                                                                                                random_state=12345, 
                                                                                                stratify=target)
# теперь разделим валидационную и тестовую
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_and_test, 
                                                                            target_valid_and_test, test_size=0.5, 
                                                                            random_state=12345, 
                                                                            stratify=target_valid_and_test)

Теперь проверим размеры получившихся выборок.

In [5]:
# обучающая
features_train.shape, target_train.shape

((1928, 4), (1928,))

In [6]:
# валидационная
features_valid.shape, target_valid.shape

((643, 4), (643,))

In [7]:
# тестовая
features_test.shape, target_test.shape

((643, 4), (643,))

Данные разделены правильно.

## Исследуйте модели

В данной работе целевой признак категориальный, поэтому решается задача классификации. А так как варианта всего два: тариф "ультра" и тариф "смарт", то классификация - бинарная.

В таком случае для обучения модели и решения задачи мы можем использовать эти алгоритмы: дерево решений, случайный лес и логистическую регрессию.

Начнем с исследования модели решающего дерева.

In [8]:
best_model_dtc = None
best_result_dtc = 0
best_depth_dtc = 0
best_split_dtc = 0
best_leaf_dtc = 0

for depth in range(1,11):
    for split in range(2, 102, 10):
        for leaf in range(2, 10):
            model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight = 'balanced',
                                                min_samples_split=split, min_samples_leaf=leaf)
            model_tree.fit(features_train, target_train)
            predictions = model_tree.predict(features_valid)
            result = accuracy_score(target_valid, predictions)
            if result > best_result_dtc:
                best_result_dtc = result
                best_model_dtc = model_tree
                best_depth_dtc = depth
                best_split_dtc = split
                best_leaf_dtc = leaf
        
print("Accuracy лучшей модели:", best_result_dtc)
print('Глубина лучшей модели:', best_depth_dtc)
print('Минимальное число объектов в узле дерева:', best_split_dtc)
print('Минимальное число объектов в листьях дерева:', best_leaf_dtc)

Accuracy лучшей модели: 0.80248833592535
Глубина лучшей модели: 6
Минимальное число объектов в узле дерева: 2
Минимальное число объектов в листьях дерева: 4


Узнали наиболее результативные гиперпараметры для решающего дерева. Теперь у нас есть модель с наилучшими в заданных диапазонах значениями.

Далее будем исследовать модель случайного леса. 

In [9]:
best_model_rfc = None
best_est_rfc = 0
best_result_rfc = 0
best_depth_rfc = 0
best_split_rfc = 0
best_leaf_rfc = 0

for est in range(1, 61, 10):
    for depth in range(1, 13):
        for split in range(2, 52, 5):
            for leaf in range (2, 12, 2):
                model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, class_weight = 'balanced', 
                                                       max_depth=depth, min_samples_split=split, min_samples_leaf=leaf)
                model_forest.fit(features_train, target_train)
                predictions = model_forest.predict(features_valid)
                result = accuracy_score(target_valid, predictions)
                if result > best_result_rfc:
                    best_result_rfc = result
                    best_model_rfc = model_forest
                    best_depth_rfc = depth
                    best_split_rfc = split
                    best_leaf_rfc = leaf
                    best_est_rfc = est
        
print('Accuracy лучшей модели:', best_result_rfc)
print('Число деревьев лучшей модели:', best_est_rfc)
print('Глубина деревьев:', best_depth_rfc)
print('Минимальное число объектов в узле дерева:', best_split_rfc)
print('Минимальное число объектов в листьях дерева:', best_leaf_rfc)

Accuracy лучшей модели: 0.8087091757387247
Число деревьев лучшей модели: 31
Глубина деревьев: 6
Минимальное число объектов в узле дерева: 17
Минимальное число объектов в листьях дерева: 4


Достигли большего значения accuracy (0.805), чем было до изменения гиперпараметров. Теперь у нас есть модель с лучшими значениями гиперпараметров (из заданного диапазона).

Далее остается создать и исследовать модель, обученную с помощью логистической регрессии.

In [10]:
best_model_lr = None
best_result_lr = 0

first_model_lr = LogisticRegression(random_state=12345)
first_model_lr.fit(features_train, target_train)
predictions = first_model_lr.predict(features_valid)
first_result_lr = accuracy_score(target_valid, predictions)

second_model_lr = LogisticRegression(random_state=12345, penalty='l1', solver='liblinear')
second_model_lr.fit(features_train, target_train)
predictions = second_model_lr.predict(features_valid)
second_result_lr = accuracy_score(target_valid, predictions)

if second_result_lr > first_result_lr:
    best_result_lr = second_result_lr
    best_model_lr = second_model_lr
    print('Лучшая модель - вторая')
else:
    best_result_lr = first_result_lr
    best_model_lr = first_model_lr
    print('Лучшая модель - первая')

print("Accuracy лучшей модели:", best_result_lr)

Лучшая модель - первая
Accuracy лучшей модели: 0.7387247278382582


Сравнили две модели: первая с гиперпараметрами по умолчанию (кроме random_state), вторая - с измененными. Использовался гиперпараметр solver='liblinear', так как, судя по документации, он больше подходит для не очень больших объемов данных. После изменения значений различных гиперпараметров выяснилось, что penalty='l1' дает лучший результат accuracy, но, тем не менее, он наименьший из всех трех.

### Вывод

Итак, создали, обучили и исследовали три вида моделей: решающее дерево, случайный лес и логистическая регрессия. 

Сначала исследовали **решающее дерево (DecisionTreeClassifier)**. В цикле создавались разные деревья с разными гиперпараметрами, обучались, делались предсказания, качество которых потом проверялось (accuracy) и сравнивалось. В итоге получили такие значения гиперпараметров как самые результативные: 

- глубина дерева (max_depth) = 6
- минимальное число объектов в узле дерева (min_samples_split) = 2
- минимальное число объектов в листьях дерева (min_samples_leaf) = 4
- **Accuracy** лучшей модели = 0.804

Далее рассматривали **случайный лес (RandomForestClassifier)**. Также использовался цикл, чтобы проверить какие результаты дает разное количество деревьев в лесе, потом - какие результаты получаются после изменения других гиперпараметров. Получили модель с такими гиперпараметрами:

- число деревьев (n_estimators) = 31
- глубина деревьев (max_depth) = 6
- минимальное число объектов в узле дерева (min_samples_split) = 17
- минимальное число объектов в листьях дерева (min_samples_leaf) = 4
- **Accuracy** лучшей модели = 0.8087

Последней стала **логистическая регрессия (LogisticRegression)**. У нее не так много гиперпараметров, которые можно легко менять. Из документации были взяты penalty='l1', solver='liblinear', так как они подходят для небольших датафреймов и бинарной классификации. Сравнили модель с этими гиперпараметрами с моделью по умолчанию. Более результативной стала измененная модель.
- **Accuracy** лучшей модели = 0.739

Таким образом самой точной моделью (после проверки на валидационной выборке) стал **случайный лес**.

## Проверьте модель на тестовой выборке

Теперь обученную модель случайного леса с самым высоким результатом accuracy можно проверить на тестовой выборке. Перед этим обучим ее дополнительно на валидационной выборке.

In [11]:
best_model_rfc.fit(pd.concat([features_valid, features_train]), pd.concat([target_valid, target_train]))
predictions = best_model_rfc.predict(features_test)
result = accuracy_score(target_test, predictions)
print('Accuracy лучшей модели случайного леса:', result)

Accuracy лучшей модели случайного леса: 0.8118195956454122


Посмотрим на другие метрики качества.

In [12]:
confusion_matrix(target_test, predictions)

array([[398,  48],
       [ 73, 124]])

Наша модель случайного леса наделала ошибок как ложноположительных (41), так и ложноотрицательных (73). Но правильных ответов, в целом, большинство.

In [13]:
precision_score(target_test, predictions)

0.7209302325581395

Полнота - метрика качества классификации, показывает, какова доля истинно-положительных ответов среди всех, у которых истинная метка 1.
В данном случае результат довольно неплох, более 0.72.

In [14]:
recall_score(target_test, predictions)

0.6294416243654822

Точность показывает, какую долю объектов, распознанных как объекты положительного класса, мы предсказали верно. Тут результат чуть похуже, ложноположительных ответов больше чем ложноотрицательных.

In [15]:
f1_score(target_test, predictions)

0.6720867208672087

F1 мера - это среднее гармоническое полноты и точности. В целом, результат нельзя назвать плохим - более 0.67.

Лучшая модель снова показала хороший результат, Accuracy выше 0.811. Проверка пройдена.

Для данного датафрейма случайный лес хороший выбор, так как количество данных здесь небольшое. В иных случаях же он может оказаться слишком медленным, хотя он показывает хорошую точность.

## Проверка модели на адекватность

In [16]:
dc_mf = DummyClassifier() 
dc_mf.fit(features_train, target_train) 

predictions_rfc = best_model_rfc.predict(features_test)
predictions_dc = dc_mf.predict(features_test)

In [17]:
accuracy_score(target_test, predictions_rfc)

0.8118195956454122

In [18]:
accuracy_score(target_test, predictions_dc)

0.6936236391912908

Значения метрики accuracy довольно высоки в обоих случаях: и для нашей лучшей модели, и для примитивной. Стоит рассмотреть другие метрики качества.

Посмотрим на матрицу неточностей.

In [19]:
confusion_matrix(target_test, predictions_rfc)

array([[398,  48],
       [ 73, 124]])

In [20]:
confusion_matrix(target_test, predictions_dc)

array([[446,   0],
       [197,   0]])

Очевидно, примитивная модель указала больше истинно отрицательных ответов потому, что она указала все ответы как наиболее часто встречающиеся отрицательные.

Рассмотрим метрику F1 и сравним результаты.

In [21]:
result_rfc = f1_score(target_test, predictions_rfc)
result_dc = f1_score(target_test, predictions_dc)

print('Метрика F1 примитивной модели:', result_dc)
print('Метрика F1 лучшей модели случайного леса:', result_rfc)

if result_rfc > result_dc:
    print('Ура! Модель адекватна.')
else:
    print('Модель не прошла проверку на вменяемость.')

Метрика F1 примитивной модели: 0.0
Метрика F1 лучшей модели случайного леса: 0.6720867208672087
Ура! Модель адекватна.
